# Steps to follow 
**Step 1 :** download the Google cloud cli 

**Step 2 :** Install the google cloud SDK using the following command:

```bash
curl https://sdk.cloud.google.com | bash
```

**Step 3 :** Configure the Google Cloud SDK
```bash
gcloud init
```

**Step 4 :** Authenticate your Google Cloud SDK with your Google Cloud account
```bash
gcloud auth login
```
**Step 5 :** Set the project ID for your Google Cloud project
```bash
gcloud config set project <project_id>
```
**Step 6 :** Create a new Google Cloud Storage bucket
```bash
gsutil mb -p <project_id> gs://<bucket_name>
```
**Step 7 :** Upload your local data to the Google Cloud Storage bucket
```bash
gsutil cp -r <local_data_dir> gs://<bucket_name>
```
**Step 8 :** Create a new Google Cloud Machine Learning Engine (ML Engine) cluster
```bash
gcloud ml-engine clusters create <cluster_name> --num-nodes 3
```
**Step 9 :** Set the Google Cloud ML Engine region
```bash
gcloud config set ml.googleapis.com/region <region>
```
**Step 10 :** Build and push the Docker image
```bash
docker build -t gcr.io/<project_id>/<cluster_name>:latest.
```
**Step 11 :** Deploy the trained model to Google Cloud ML Engine
```bash
gcloud ml-engine models create <model_name> --regions <region>
```
```bash
gcloud ml-engine versions create v1 --model <model_name> --runtime-version 1.15 --image gcr.io/<project_id>/<cluster_name>:latest
```
**Step 12 :** Create a new Google Cloud Run service
```bash
gcloud run deploy --image gcr.io/<project_id>/<cluster_name>:latest --platform managed --region <region> --allow-unauthenticated --update-env-vars PROJECT_NAME=<project_name>
```
**Step 13 :** Test the deployed model
```bash
gcloud run services describe <service_name> --platform managed --region <region> --format=json | jq -r '.status.url'
```
**Step 14 :** Delete the Google Cloud resources
```bash
gcloud run services delete <service_name> --platform managed --region <region> --quiet
```
**Step 15 :** Delete the Google Cloud Storage bucket
```bash
gsutil rm -r gs://<bucket_name>
```
**Step 16 :** Delete the Google Cloud ML Engine cluster
```bash
gcloud ml-engine clusters delete <cluster_name> --quiet
```
**Step 17 :** Delete the Google Cloud Run service
```bash
gcloud run services delete <service_name> --platform managed --region <region> --quiet
```
**Step 18 :** Delete the Google Cloud project
```bash
gcloud projects delete <project_id> --quiet
```
**Note: This is a basic guideline and actual steps may vary based on your project requirements and the specific Google Cloud services you are using. Make sure to replace placeholders with your actual project details and configurations.**

# Set-Up Gcloud cli for project and  bucket for google cloud services

In [9]:
project_name="Hotel-Reservation-Prediction"

**Step 1 :** download the Google cloud cli 

**Step 2 :** Install the google cloud SDK using the following command:

```bash
curl https://sdk.cloud.google.com | bash
```

**Step 3 :** Configure the Google Cloud SDK
```bash
gcloud init
```

**Step 4 :** Authenticate your Google Cloud SDK with your Google Cloud account
```bash
gcloud auth login
```
**Step 5 :** Set the project ID for your Google Cloud project
```bash
gcloud config set project <project_id>
```
**Step 6 :** Create a new Google Cloud Storage bucket
```bash
gsutil mb -p <project_id> gs://<bucket_name>
```
**Step 7 :** Upload your local data to the Google Cloud Storage bucket
```bash
gsutil cp -r <local_data_dir> gs://<bucket_name>
```
step 8: create service account

To create a service account, follow these steps:

1. Go to the Google Cloud Console.
2. Go to IAM & Admin.
3. Click on "Create service account".
4. Give the service account a name and select the role **"Storage Admin"** and  **"Storage Object Viewer"**.
5. Go to the bucket you want to grant access to. And click on 3dots to select **Edit Access**  
6. Under on **Add Principal** add **your service account** and Assign Roles **"Storage Admin"** and  **"Storage Object Viewer"** and click on **Save** 
7. Goto your service account created befor in Iam & Admin Section, Click on **3dots:**  and **add key** to download key in JSON key file for the service account.

``` bash 
set GOOGLE_APPLICATION_CREDENTIALS="C:\\path to the Json file downloaded before using above step"

```

In [2]:
import os

In [11]:
%pwd

'e:\\Mlops_Udemy\\Hotel Reservation Prediction'

In [ ]:
#os.chdir("../")

In [12]:
%pwd

'e:\\Mlops_Udemy\\Hotel Reservation Prediction'

## config/config.yaml

In [ ]:
%%writefile -a config/config.yaml

data_ingestion:
  bucket_name : "my_bucket9789"
  bucket_file_name : "Hotel_Reservations.csv"
  train_ratio : 0.8


Overwriting config/config.yaml


In [ ]:
%%writefile -a config/path_config.py

import os

########################### DATA INGESTION #########################

RAW_DIR = "artifacts/raw"
RAW_FILE_PATH = os.path.join(RAW_DIR,"raw.csv")
TRAIN_FILE_PATH = os.path.join(RAW_DIR,"train.csv")
TEST_FILE_PATH = os.path.join(RAW_DIR,"test.csv")

CONFIG_PATH = "config/config.yaml"


Appending to config/path_config.py


# Data Ingestion 

In [1]:
%%writefile -a src/data_ingestion.py
import os
import pandas as pd
from google.cloud import storage
from sklearn.model_selection import train_test_split
from src.logger import get_logger
from src.custom_exception import CustomException
from config.paths_config import *
from utils.common_functions import read_yaml

logger = get_logger(__name__)


Appending to src/data_ingestion.py


In [2]:
%%writefile -a src/data_ingestion.py
class DataIngestion:
    def __init__(self,config):
        self.config = config["data_ingestion"]
        self.bucket_name = self.config["bucket_name"]
        self.file_name = self.config["bucket_file_name"]
        self.train_test_ratio = self.config["train_ratio"]

        os.makedirs(RAW_DIR , exist_ok=True)

        logger.info(f"Data Ingestion started with {self.bucket_name} and file is  {self.file_name}")

    def download_csv_from_gcp(self):
        try:
            client = storage.Client()
            bucket = client.bucket(self.bucket_name)
            blob = bucket.blob(self.file_name)

            blob.download_to_filename(RAW_FILE_PATH)

            logger.info(f"CSV file is sucesfully downloaded to {RAW_FILE_PATH}")

        except Exception as e: 
            logger.error("Error while downloading the csv file")
            raise CustomException("Failed to downlaod csv file ", e)
        
    def split_data(self):
        try:
            logger.info("Starting the splitting process")
            data = pd.read_csv(RAW_FILE_PATH)
            train_data , test_data = train_test_split(data , test_size=1-self.train_test_ratio , random_state=42)

            train_data.to_csv(TRAIN_FILE_PATH)
            test_data.to_csv(TEST_FILE_PATH)

            logger.info(f"Train data saved to {TRAIN_FILE_PATH}")
            logger.info(f"Test data saved to {TEST_FILE_PATH}")
        
        except Exception as e:
            logger.error("Error while splitting data")
            raise CustomException("Failed to split data into training and test sets ", e)
        
    def run(self):

        try:
            logger.info("Starting data ingestion process")

            self.download_csv_from_gcp()
            self.split_data()

            logger.info("Data ingestion completed sucesfully")
        
        except CustomException as ce:
            logger.error(f"CustomException : {str(ce)}")
        
        finally:
            logger.info("Data ingestion completed")

if __name__ == "__main__":
    data_ingestion = DataIngestion(read_yaml(CONFIG_PATH))
    data_ingestion.run()



Appending to src/data_ingestion.py


# Training Pipeline

In [ ]:
%%writefile -a pipeline/traing_pipeline.py
from utils.common_functions import read_yaml
from config.paths_config import *

Appending to pipeline/traing_pipeline.py


In [ ]:
%%writefile -a pipeline/traing_pipeline.py


if __name__=="__main__":
    ### 1. Data Ingestion
    from src.data_ingestion import DataIngestion
    data_ingestion = DataIngestion(read_yaml(CONFIG_PATH))
    data_ingestion.run()
    ############################################################




Appending to pipeline/traing_pipeline.py
